In [10]:
import torch
import torchvision
import clip
from PIL import Image
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")



Using cuda device


In [11]:
clip_model_type = "ViT-B/32"
csv_path = "./Charades/Charades_v1_train.csv"
video_path = "./video" 
pt_path = "./video_tensors"
frames_no = 40 
num_video_batch = 100
model, preprocess = clip.load(clip_model_type, device=device, jit=False)

In [3]:
#extract frames
def extract_frames(video_file):
    video, audio, info = torchvision.io.read_video(video_file, output_format='TCHW')

    num_frames = len(video)
    frame_rate = info["video_fps"]
    interval = int(frame_rate * 60 / frames_no)
    frames = []
    to_pil = torchvision.transforms.ToPILImage()
    for i in range(0, num_frames, interval):
      frames.append(to_pil(video[i].squeeze(0)))    

    return frames

In [4]:
#encode using clip image encoder and stack the tensors alon a new dimension
def preprocess_frames(frames):

    tensors = []

    for i in range(0,len(frames)): 
        frame = preprocess(frames[i]).unsqueeze(0).to(device)
        tensors.append(frame)

    preprocess_batch = torch.cat(tensors, dim = 0)
    preprocess_batch = preprocess_batch.to(device)

    return preprocess_batch


In [5]:
# Define a function to concatenate tensors 
def CLIPtokenise(tensors):
    #the input tensors are 3d stacks (lets say x,y,z)
    #function returns a tensor of shape (x, y, sum(z)) and a list of z values

    # get the z values for each tensor
    z_values = [t.shape[0] for t in tensors]

    joined = torch.cat(tensors, dim=0)

    with torch.no_grad():
            image_features = model.encode_image(joined)

    image_features.to(device)
    print(image_features.shape)

    #gets z values and splits them back to video stacks
    split = torch.split(image_features, z_values, dim=0)

    return split




In [6]:
# load captions and video names
df = pd.read_csv(csv_path)
# df_new = pd.read_csv(csv_path)
# df = df_new.head(16)
ids = df["id"]
errors = []

# loop over the ids with a batch size step
for i in tqdm(range(0, len(ids), num_video_batch)):
    # get the current batch of ids
    batch_ids = ids[i:i+num_video_batch].copy()
    batch_ids = batch_ids.reset_index(drop=True)

    batch = []
    temp_errors = []

    # loop over the batch ids
    for j in range(len(batch_ids)):
        try:
            # get the video file name with the corresponding id
            # video_file = f"{video_path}/{batch_ids[j]}.mp4"
            video_file = os.path.join(video_path, batch_ids[j] + ".mp4")
            batch.append(preprocess_frames(extract_frames(video_file)))
            temp_errors.append(True)
        except Exception as e:
            temp_errors.append(False)
            errors.append(f"video {batch_ids[j]} error {e}")
        
        
    tokenised_batch = CLIPtokenise(batch)

    l = 0
    for k in range(len(batch_ids)):
        if temp_errors[k]:
            torch.save(tokenised_batch[l], f"{pt_path}/{batch_ids[k]}.pt")
            l=l+1

        


  0%|          | 0/80 [00:00<?, ?it/s]c:\Users\dhavi\anaconda3\envs\clipcap\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


torch.Size([2050, 512])


  1%|▏         | 1/80 [02:02<2:40:48, 122.14s/it]

torch.Size([2053, 512])


  2%|▎         | 2/80 [04:02<2:37:35, 121.23s/it]

...

 98%|█████████▊| 78/80 [4:15:53<05:32, 166.31s/it]

torch.Size([2119, 512])


 99%|█████████▉| 79/80 [4:18:27<02:42, 162.67s/it]

torch.Size([1707, 512])


100%|██████████| 80/80 [4:20:29<00:00, 195.36s/it]


In [ ]:
df = pd.read_csv(csv_path)
# Initialize empty lists to store the embeddings and captions
all_embeddings = []
all_captions = []


for i in tqdm(range(len(df["id"]))):
  path = os.path.join(pt_path, df.loc[i, "id"] + ".pt")
  #load the .pt files
  data = torch.load(path)
  # Extract the clip embedding and caption
  clip_embedding = data
  caption = df.loc[i, 'descriptions']
  # Append them to the lists
  all_embeddings.append(clip_embedding)
  all_captions.append(caption)

# Concatenate the embeddings into a single tensor
clip_embedding = torch.cat(all_embeddings, dim=0)

# Save the final dictionary as a pickle file
output_path = "./clip_caption/clip_caption.pkl"
with open(output_path, 'wb') as f:
  pickle.dump({"clip_embedding": clip_embedding, "captions": all_captions}, f)


In [7]:
print(errors)


[]


In [12]:
#Comparison between batch processing and processing frame by frame
video, audio, info = torchvision.io.read_video("./video/S6MPZ.mp4", output_format='TCHW')
to_pil = torchvision.transforms.ToPILImage()
image = preprocess(to_pil(video[0].squeeze(0))).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)


tensor = torch.load("./video_tensors/S6MPZ.pt")

print(tensor)
cos_sim = torch.nn.functional.cosine_similarity(image_features, tensor[0].unsqueeze(0)) 
euc_dist = torch.nn.functional.pairwise_distance(image_features, tensor[0].unsqueeze(0))

print(cos_sim)
print(euc_dist)


c:\Users\dhavi\anaconda3\envs\clipcap\Lib\site-packages\torchvision\io\video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


tensor([[ 0.0497,  0.1740, -0.0112,  ...,  1.2744, -0.3362,  0.2883],
        [ 0.0991,  0.2061,  0.0660,  ...,  1.6748, -0.4888,  0.2273],
        [ 0.0687,  0.2087, -0.2421,  ...,  1.5742, -0.4629,  0.1131],
        ...,
        [ 0.2361,  0.0197,  0.1917,  ...,  1.0273, -0.1343,  0.1251],
        [ 0.2163,  0.0526,  0.2136,  ...,  0.9785, -0.1228,  0.2971],
        [-0.0357,  0.1965, -0.0172,  ...,  1.1807, -0.1632,  0.2786]],
       device='cuda:0', dtype=torch.float16)
tensor([1.], device='cuda:0', dtype=torch.float16)
tensor([0.0168], device='cuda:0', dtype=torch.float16)
